In [ ]:
%%time
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from statistics import mean 
import time
import torch

import sys
script_path = os.path.abspath("pyscripts")
if script_path not in sys.path:
    sys.path.append(script_path)
from utils import *
from load_data import *
from lstm import *

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# 1. Parameters

In [2]:
# time sequence
train_window = 48
input_size = 1
output_size = 24
hidden_size = 48
num_layers = 2
learning_rate = 0.005

# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Read Data

In [3]:
obs = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/Geotab_Obs_Ratio05.npy')
# swap rows and columns, obs = np.swapaxes(obs, 0, 1)
obs = obs.T
times = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/Geotab_Obs_TS_Ratio05.npy')
times = [datetime.utcfromtimestamp(x.tolist()/1e9) for x in times]
stations = pd.read_csv('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/selected_Geotab_stations_Ratio05.csv')
stations = stations[stations.columns[[3,1,2,4]]]

weather_station_2015_2020 = pd.read_csv("/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_station_2015_2020.csv")
weather_time_2015_2020 = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_time_2015_2020.npy', allow_pickle=True)

weather_obs_2015_2020 = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_obs_2015_2020.npy')
weather_obs_2010_2020 = np.load('/home/graduate/fbx5002/disk10TB/IEE/IoT_Weather_Data/weather_obs_2010_2020.npy')

In [4]:
print('geotab length:', len(times))

geotab length: 8784


In [5]:
stations5 = set(stations.sort_values('missing')[:455]['geohash'])
stations10= set(stations.sort_values('missing')[:1650]['geohash'])
stations15= set(stations.sort_values('missing')[:2107]['geohash'])
stations20= set(stations.sort_values('missing')[:2562]['geohash'])
# stations25= set(stations.sort_values('missing')[:2949]['geohash'])
# stations30= set(stations.sort_values('missing')[:3220]['geohash'])
# stations35= set(stations.sort_values('missing')[:3571]['geohash'])
# stations40= set(stations.sort_values('missing')[:3943]['geohash'])
# stations45= set(stations.sort_values('missing')[:4339]['geohash'])
# stations50 = set(stations['geohash'])
wu_stations = set(weather_station_2015_2020['station'])

In [ ]:
after_missingratio = list(map(lambda x: sum(np.isnan(x))/8784, obs))
print("The data shape of IoT and Weather_Underground data are %s, %s" %(obs.shape, weather_obs_2015_2020.shape))
print("The time length of IoT and Weather_Underground data are %s, %s" % (len(times), len(weather_time_2015_2020)))
print("Time window of IoT and Weather_Underground data are [%s, %s], [%s, %s]" % 
      (times[0].strftime("%Y%m%d%H"), times[-1].strftime("%Y%m%d%H"), 
       weather_time_2015_2020[0].strftime("%Y%m%d"), weather_time_2015_2020[-1].strftime("%Y%m%d")))
print("Original Minimum and Maximum Missing Data Ratio for IoT Stations are %s, %s" % (min(stations['missing']), max(stations['missing'])))
print("After Interpolation, Minimum and Maximum Missing Data Ratio for IoT Stations are %s, %s" % (min(after_missingratio), max(after_missingratio)))
print(stations.head(5)); print(weather_station_2015_2020.head(5))

In [ ]:
# select days for testing
test_times = []
for _time in times:
    if _time.hour > 0:
        continue
    test_times.append(datetime(_time.year, _time.month, _time.day, 0 , 0))
random.seed(7)
seq = [i for i in range(len(test_times))]
random.shuffle(seq) # inplace shuffle
test_times = np.array(sorted([test_times[i] for i in seq[:30]]))
print(test_times)

# 3. Create infos dict and Calculate min and max

In [ ]:
infos = {}
for i, station in enumerate(stations['geohash']):
    infos[station] = obs[i]
print(list(infos.items())[:2])

# operations below will in-place rewrite infos variable
norm_min = []
norm_max = []
for key, value in infos.items():
    norm_min.append(min(value))
    norm_max.append(max(value))
# len(list(infos_filtered.items())[0][1]) # this is the station we used for the single station training
norm_min = min(norm_min)
norm_max = max(norm_max)
print("The minimum and maximal temperatures for IoT stations are %.2f, %.2f" %(norm_min, norm_max))

# Observation: the higher interpolation rate, more similar/correlated that two stations data are
# why 'dr5nw3r' looks same as 'dr5nw92'
'dr5nw3r' in stations20

In [ ]:
infos_w = {}
for i, station in enumerate(weather_station_2015_2020['station']):
    infos_w[station] = weather_obs_2015_2020[i]
print(list(infos_w.items())[:2])

# operations below will in-place rewrite infos variable
norm_min_w = []
norm_max_w = []
for key, value in infos_w.items():
    norm_min_w.append(min(value))
    norm_max_w.append(max(value))
# len(list(infos_filtered.items())[0][1]) # this is the station we used for the single station training
norm_min_w = min(norm_min_w)
norm_max_w = max(norm_max_w)
print("The minimum and maximal temperatures for Weather Underground stations are %.2f, %.2f" %(norm_min_w, norm_max_w))

In [ ]:
norm_min = min(norm_min, norm_min_w)
norm_max = max(norm_max, norm_max_w)
print(norm_min, norm_max)

# 4. Split data for test_day and train_day for IoT stations

### key point
- only need to make sure the 24 hours output for test_times not seen in the training dataset
- the 48 hours input of test_times can be seen in the training dataset
- weather underground stations all for training

In [ ]:
test_time_range = []
for testtime in test_times:
    start = testtime - timedelta(hours = train_window)
    end = testtime + timedelta(hours = output_size-1) # include itself
    if end > times[-1]:
        continue
    test_time_range.append((start, end))
    
test_time_index = sorted([(times.index(x), times.index(y)) for x, y in test_time_range])
removed_time_index = sorted([(times.index(x)+train_window, times.index(y)) for x, y in test_time_range])
print(test_time_index[:5])
print(removed_time_index[:5])

In [ ]:
test_data = {}
train_data = {}
for key, data in list(infos.items()):
    test, train = [], []
    for test_time in test_time_index:
        test.append(data[test_time[0]:(test_time[1]+1)])
    
    for i in range(len(removed_time_index)):
        if i ==0:
            end = removed_time_index[0][0]
            train.append(data[:end])   
        else:
            start = removed_time_index[i-1][1]+1
            end = removed_time_index[i][0]
            train.append(data[start:end])
    # add elements after last removed_time_index
    start = removed_time_index[len(removed_time_index)-1][1]+1
    train.append(data[start:])
    # change to nparray
    train_data[key] = train
    test_data[key] = test

#len(infos['dr72rzt'][0]) == len(train_data['dr72rzt'][0]) + len(test_data['dr72rzt'][0]) 
l1 = sum([len(x) for x in train_data['dr72rzt']])
l2 = sum([y-x+1 for x, y in removed_time_index])
print(l1, l2)
l1+l2 == len(infos['dr72rzt'])

# 5. Prepare training and test tensors

In [ ]:
start = time.time()
train_data.update(infos_w)
train_tensors = Dataset(train_data, (norm_min, norm_max), train_window, output_size)
end = time.time()
print(end-start)
print("Training input and output for a IoT station: %s, %s" % (train_tensors[0][0].shape, train_tensors[0][1].shape))
print("Validation input and output for a IoT station: %s, %s" % (train_tensors[0][2].shape, train_tensors[0][3].shape))
idx = train_tensors.keys.index('KNJCLIFF7')
print("Training input and output for a weather station: %s, %s" % (train_tensors[idx][0].shape, train_tensors[idx][1].shape))
print("Validation input and output for a weather station: %s, %s" % (train_tensors[idx][2].shape, train_tensors[idx][3].shape))

In [ ]:
start = time.time()
test_tensors = DatasetTestDays(test_data, (norm_min, norm_max), train_window)
end = time.time()
print(end-start)
print("Testing input and output for one IoT station: %s, %s" % (test_tensors[0][0].shape, test_tensors[0][1].shape))

In [ ]:
# three selected days have na values in their observations which cannot be predicted
missing_days = np.unique(test_tensors.missing)
print(missing_days)
test_times = np.delete(test_times, missing_days)

In [ ]:
db = {'train': train_tensors, 'test': test_tensors, 'selected_days': test_times}
torch.save(db, './checkpoint/IoT_final/TensorData/Torch_db5WU')

# 6.2 Train LSTM 

In [ ]:
loaded = torch.load('./checkpoint/IoT_final/TensorData/Torch_db5WU')
train_tensors = loaded['train']
test_tensors = loaded['test']
test_times = loaded['selected_days']
norm_min, norm_max = train_tensors.min, train_tensors.max

In [6]:
# To compute the number of trainable parameters
# dir() or __dir__:return an alphabetized list of names comprising (some of) the attributes of the given object, and of attributes reachable from it
# dir() or __dir__:return an alphabetized list of names comprising (some of) the attributes of the given object, and of attributes reachable from it
# print(dir(list(model.parameters())[0]))
_, model, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
totalTrainable = []
for name, p in model.named_parameters():
    print(name, p.shape)
    # total number of trainable parameters
    if p.requires_grad:
        totalTrainable.append(p.numel())
    else:
        totalTrainable.append(None)
        
print(totalTrainable)

num_epochs = 101
epoch_interval = 2

AttributeError: 'LSTM' object has no attribute 'to'

# IoT Only

In [ ]:
# initialize and train the model for 5.5%
checkpoint_format = 'checkpoint/IoT_final/checkpoint-5.5-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, device, stationlist = stations5)

In [ ]:
# initialize and train the model for 10%
checkpoint_format = 'checkpoint/IoT_final/checkpoint-10-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, device, stationlist = stations10)

In [ ]:
# initialize and train the model for 15%
checkpoint_format = 'checkpoint/IoT_final/checkpoint-15-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, device, stationlist = stations15)

In [ ]:
# initialize and train the model for 20%
checkpoint_format = 'checkpoint/IoT_final/checkpoint-20-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
         device, stationlist = stations20)

In [ ]:
# # initialize and train the model for 25%
# checkpoint_format = 'checkpoint/IoT_final/checkpoint-25-{epoch}.pth.tar'
# loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
# Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
#          device, stationlist = stations25)

# # initialize and train the model for 30%
# checkpoint_format = 'checkpoint/IoT_final/checkpoint-30-{epoch}.pth.tar'
# loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
# Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
#          device, stationlist = stations30)

# # initialize and train the model for 35%
# checkpoint_format = 'checkpoint/IoT_final/checkpoint-35-{epoch}.pth.tar'
# loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
# Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
#          device, stationlist = stations35)

# # initialize and train the model for 40%
# checkpoint_format = 'checkpoint/IoT_final/checkpoint-40-{epoch}.pth.tar'
# loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
# Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
#          device, stationlist = stations40)

# # initialize and train the model for 45%
# checkpoint_format = 'checkpoint/IoT_final/checkpoint-45-{epoch}.pth.tar'
# loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
# Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
#          device, stationlist = stations45)

# # initialize and train the model for 50%
# checkpoint_format = 'checkpoint/IoT_final/checkpoint-50-{epoch}.pth.tar'
# loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
# Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format, 
#          device, stationlist = stations50)

## Add 5 WU

In [ ]:
# initialize and train the model for 5.5%IoT+5WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-5.5-IoT-5-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations5.union(wu_stations))

In [ ]:
# initialize and train the model for 10%IoT+5WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-10-IoT-5-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations10.union(wu_stations))

In [ ]:
# initialize and train the model for 15%IoT+5 WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-15-IoT-5-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations15.union(wu_stations))

In [ ]:
# initialize and train the model for 20%IoT+5 WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-20-IoT-5-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations20.union(wu_stations))

## Add 10 WU

In [ ]:
# initialize and train the model for 5.5%IoT+10WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-5.5-IoT-10-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations5.union(wu_stations))

In [ ]:
# initialize and train the model for 10%IoT+10WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-10-IoT-10-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations10.union(wu_stations))

In [ ]:
# initialize and train the model for 15%IoT+10WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-15-IoT-10-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations15.union(wu_stations))

In [ ]:
# initialize and train the model for 20%IoT+10WU
checkpoint_format = 'checkpoint/IoT_final/checkpoint-20-IoT-10-W-{epoch}.pth.tar'
loss_func, model, optimizer = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device) 
Training(loss_func, model, optimizer, train_tensors, num_epochs, epoch_interval, checkpoint_format,
         device, stationlist = stations20.union(wu_stations))

# 7. Predict and Statistic Analysis_SplitByDay

In [ ]:
loaded = torch.load('./checkpoint/IoT_final/TensorData/Torch_db5WU')
train_tensors = loaded['train']
test_tensors = loaded['test']
test_times = loaded['selected_days']
norm_min, norm_max = train_tensors.min, train_tensors.max

print(norm_min, norm_max)

print("Training input and output for a IoT station: %s, %s" % (train_tensors[0][0].shape, train_tensors[0][1].shape))
print("Validation input and output for a IoT station: %s, %s" % (train_tensors[0][2].shape, train_tensors[0][3].shape))
idx = train_tensors.keys.index('KNJCLIFF7')
print("Training input and output for a weather station: %s, %s" % (train_tensors[idx][0].shape, train_tensors[idx][1].shape))
print("Validation input and output for a weather station: %s, %s" % (train_tensors[idx][2].shape, train_tensors[idx][3].shape))

In [ ]:
# read the saved model and use the best one for predicting
checkpoint5 = torch.load('./checkpoint/IoT_final/checkpoint-5.5-{epoch}.pth.tar'.format(epoch=60))
checkpoint10 = torch.load('./checkpoint/IoT_final/checkpoint-10-{epoch}.pth.tar'.format(epoch=23))
checkpoint15 = torch.load('./checkpoint/IoT_final/checkpoint-15-{epoch}.pth.tar'.format(epoch=8))
checkpoint20 = torch.load('./checkpoint/IoT_final/checkpoint-20-{epoch}.pth.tar'.format(epoch=10))
# checkpoint25 = torch.load('./checkpoint/IoT_final/checkpoint-25-{epoch}.pth.tar'.format(epoch=4))
# checkpoint30 = torch.load('./checkpoint/IoT_final/checkpoint-30-{epoch}.pth.tar'.format(epoch=6))
# checkpoint35 = torch.load('./checkpoint/IoT_final/checkpoint-35-{epoch}.pth.tar'.format(epoch=7))
# checkpoint40 = torch.load('./checkpoint/IoT_final/checkpoint-40-{epoch}.pth.tar'.format(epoch=9))
# checkpoint45 = torch.load('./checkpoint/IoT_final/checkpoint-45-{epoch}.pth.tar'.format(epoch=6))
# checkpoint50 = torch.load('./checkpoint/IoT_final/checkpoint-50-{epoch}.pth.tar'.format(epoch=3))

checkpoint5IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-5.5-IoT-10-W-{epoch}.pth.tar'.format(epoch=19))
checkpoint10IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-10-IoT-10-W-{epoch}.pth.tar'.format(epoch=11))
checkpoint15IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-15-IoT-10-W-{epoch}.pth.tar'.format(epoch=4))
checkpoint20IoT10WU = torch.load('./checkpoint/IoT_final/checkpoint-20-IoT-10-W-{epoch}.pth.tar'.format(epoch=7))

checkpoint5IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-5.5-IoT-5-W-{epoch}.pth.tar'.format(epoch=18))
checkpoint10IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-10-IoT-5-W-{epoch}.pth.tar'.format(epoch=9))
checkpoint15IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-15-IoT-5-W-{epoch}.pth.tar'.format(epoch=5))
checkpoint20IoT5WU = torch.load('./checkpoint/IoT_final/checkpoint-20-IoT-5-W-{epoch}.pth.tar'.format(epoch=7))

# initialize and train the model
_, model5, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model10, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model15, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model20, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
# _, model25, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
# _, model30, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
# _, model35, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
# _, model40, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
# _, model45, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
# _, model50, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)

_, model5IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model10IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model15IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model20IoT10WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)

_, model5IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model10IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model15IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)
_, model20IoT5WU, _ = initial_model(input_size, output_size, hidden_size, num_layers, learning_rate, device)

model5.load_state_dict(checkpoint5['bestLossModel'])
model10.load_state_dict(checkpoint10['bestLossModel'])
model15.load_state_dict(checkpoint15['bestLossModel'])
model20.load_state_dict(checkpoint20['bestLossModel'])
# model25.load_state_dict(checkpoint25['bestLossModel'])
# model30.load_state_dict(checkpoint30['bestLossModel'])
# model35.load_state_dict(checkpoint35['bestLossModel'])
# model40.load_state_dict(checkpoint40['bestLossModel'])
# model45.load_state_dict(checkpoint45['bestLossModel'])
# model50.load_state_dict(checkpoint50['bestLossModel'])

model5IoT10WU.load_state_dict(checkpoint5IoT10WU['bestLossModel'])
model10IoT10WU.load_state_dict(checkpoint10IoT10WU['bestLossModel'])
model15IoT10WU.load_state_dict(checkpoint15IoT10WU['bestLossModel'])
model20IoT10WU.load_state_dict(checkpoint20IoT10WU['bestLossModel'])
 
model5IoT5WU.load_state_dict(checkpoint5IoT5WU['bestLossModel'])
model10IoT5WU.load_state_dict(checkpoint10IoT5WU['bestLossModel'])
model15IoT5WU.load_state_dict(checkpoint15IoT5WU['bestLossModel'])
model20IoT5WU.load_state_dict(checkpoint20IoT5WU['bestLossModel'])

In [ ]:
# Save into npy
tests = ['model5', 'model10', 'model15', 'model20', 
         'model25', 'model30', 'model35', 'model40', 'model45', 'model50',
         'model5IoT5WU', 'model10IoT5WU', 'model15IoT5WU', 'model20IoT5WU',
        'model5IoT10WU', 'model10IoT10WU', 'model15IoT10WU', 'model20IoT10WU']
models = [model5, model10, model15, model20, 
          model25, model30, model35, model40, model45, model50,
          model5IoT5WU, model10IoT5WU, model15IoT5WU, model20IoT5WU,
         model5IoT10WU, model10IoT10WU, model15IoT10WU, model20IoT10WU]
stations_ = [stations5, stations10, stations15, stations20, stations5, stations10, stations15, stations20]

for i in range(len(tests)):
    output_folder = 'IoT_Weather_Data/output/testOnStation5/'+tests[i]+'/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    test_pred_orig_dict = predict(models[i], test_tensors, (norm_min, norm_max), device, stationlist = stations5)

    # test_stations are not shuffled
    outputStations = stations[stations['geohash'].isin(list(test_pred_orig_dict.keys()))]
    list(outputStations['geohash'])== list(test_pred_orig_dict.keys())

    outputStations.to_csv(output_folder+'stations.csv', index=False)
    np.save(output_folder+'selectedDates.npy', test_times)
    np.save(output_folder+'predictions.npy', np.array([x[0].numpy() for x in list(test_pred_orig_dict.values())]))
    np.save(output_folder+'observations.npy', np.array([x[1].numpy() for x in list(test_pred_orig_dict.values())]))

In [ ]:
checkpoints = [checkpoint5, checkpoint10, checkpoint15, checkpoint20]
stations_ = [stations5, stations10, stations15, stations20]
fig, axs = plt.subplots(1, 4, figsize=(20,4))
for i, ax in enumerate(axs):   
    ax.plot(meanLossEpoch(checkpoints[i]['train_loss'], len(stations_[i])))
    ax.plot(meanLossEpoch(checkpoints[i]['test_loss'], len(stations_[i])))
    ax.set_ylim([0, 0.0025])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend(["Training", "Validation"])
plt.savefig('LossThreeRatios.jpeg', dpi=300)
plt.show 

In [ ]:
checkpoints = [checkpoint5IoTWU, checkpoint10IoTWU, checkpoint15IoTWU, checkpoint20IoTWU]
stations_ = [stations5, stations10, stations15, stations20]
fig, axs = plt.subplots(1, 4, figsize=(20,4))
for i, ax in enumerate(axs):   
    ax.plot(meanLossEpoch(checkpoints[i]['train_loss'], len(stations_[i])))
    ax.plot(meanLossEpoch(checkpoints[i]['test_loss'], len(stations_[i])))
    ax.set_ylim([0, 0.0025])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend(["Training", "Validation"])
plt.savefig('LossThreeRatios_IOT_WU.jpeg', dpi=300)
plt.show 

## Compare four models on 5.5% stations

In [ ]:
test_pred_orig_dict5 = predict(model5, test_tensors, (norm_min, norm_max), device, stationlist = stations5)
test_pred_orig_dict10 = predict(model10, test_tensors, (norm_min, norm_max), device, stationlist = stations5)
test_pred_orig_dict15 = predict(model15, test_tensors, (norm_min, norm_max), device, stationlist = stations5)
test_pred_orig_dict20 = predict(model20, test_tensors, (norm_min, norm_max), device, stationlist = stations5)

# calculate root mean squared error
testScores_stations5, test_min5, test_mean5, test_max5 = stat_scores(test_pred_orig_dict5)
testScores_stations10, test_min10, test_mean10, test_max10 = stat_scores(test_pred_orig_dict10)
testScores_stations20, test_min20, test_mean20, test_max20= stat_scores(test_pred_orig_dict20)

In [ ]:
test_pred_orig_dict = predict(modelIoTWU, test_tensors, (norm_min, norm_max), device, stationlist = stations5)
# calculate root mean squared error
testScores_stations, test_min, test_mean, test_max = stat_scores(test_pred_orig_dict)

In [ ]:
# # ##### plot baseline and predictions for test station having max RMSE
# # fig, axs = plt.subplots(5, 5, sharex=True, sharey=True, figsize=(14,10))
# # i  =0
# # for row in axs:
# #     for col in row:
# #         col.plot(test_pred_orig_dict[test_mean][1][i]) # original
# #         col.plot(test_pred_orig_dict[test_mean][0][i], '--') # predicted
# #         i+=1

# errors_rmse = list(map(lambda x, y : math.sqrt(mean_squared_error(x, y)), 
#                                 test_pred_orig_dict[test_mean][0],test_pred_orig_dict[test_mean][1]))
# errors_mae = list(map(lambda x, y : mean(abs(x- y).numpy()), 
#                                 test_pred_orig_dict[test_mean][0],test_pred_orig_dict[test_mean][1]))
# print(sorted(errors_rmse, reverse=True))
# print(sorted(errors_mae, reverse=True))
# print("\n")
# print("Average RMSE over 27 selected days is %s" % mean(errors_rmse))
# idx = [i for i, error in enumerate(errors_rmse) if error > 1.5*mean(errors_rmse)]
# outlier_times = sorted([test_times[i] for i in idx])
# print(outlier_times)

# weights = np.ones_like(errors_rmse) / (len(errors_rmse))
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,5))
# ax1.hist(errors_rmse, bins = 20, range=(0.5, 8), edgecolor = 'black', weights= weights)
# ax1.set_xlabel('Average RMSE for Each Selected Day')
# ax1.set_ylabel('Percentage')
# ax1.set_ylim(0, 0.3)
# ax2.hist(errors_mae, bins = 20, range=(0.5, 8), edgecolor = 'black', weights= weights)
# ax2.set_xlabel('Average MAE for Each Selected Day')
# ax2.set_ylabel('Percentage')
# plt.savefig('HistRMSEDays.jpeg', dpi=300)
# plt.show()

# testScores_stations = dict()
# idx = [error < 1.5*mean(errors_rmse) for error in errors_rmse]
# testScores_stations, test_min_, test_mean_, test_max_  = stat_scores(test_pred_orig_dict, idx)
# print("Test Stations with min, mean and max RMSE are %s, %s, %s" % (test_min_, test_mean_, test_max_))

# # mae over predicted time points for test station with mean error
# over_time_mae = np.mean(abs(test_pred_orig_dict[test_mean][0][idx] - test_pred_orig_dict[test_mean][1][idx]).numpy(), axis =0)
# print(over_time_mae)
# plt.plot(over_time_mae)
# plt.show()

## Compare model10 and model IoTWU to predict on 10% stations

In [ ]:
test_pred_orig_dict10 = predict(model10, test_tensors, (norm_min, norm_max), device, stationlist = stations10)
# calculate root mean squared error
testScores_stations10, test_min10, test_mean10, test_max10 = stat_scores(test_pred_orig_dict10)

test_pred_orig_dict = predict(modelIoTWU, test_tensors, (norm_min, norm_max), device, stationlist = stations10)
# calculate root mean squared error
testScores_stations, test_min, test_mean, test_max = stat_scores(test_pred_orig_dict)

In [ ]:
errors_rmse = list(map(lambda x, y : math.sqrt(mean_squared_error(x, y)), 
                                test_pred_orig_dict[test_mean][0],test_pred_orig_dict[test_mean][1]))
errors_mae = list(map(lambda x, y : mean(abs(x- y).numpy()), 
                                test_pred_orig_dict[test_mean][0],test_pred_orig_dict[test_mean][1]))
print(sorted(errors_rmse, reverse=True))
print(sorted(errors_mae, reverse=True))
print("\n")
print("Average RMSE over 27 selected days is %s" % mean(errors_rmse))
idx = [i for i, error in enumerate(errors_rmse) if error > 1.5*mean(errors_rmse)]
outlier_times = sorted([(test_times[i], errors_rmse[i]) for i in idx])
print(idx, outlier_times)

In [ ]:
weights = np.ones_like(errors_rmse) / (len(errors_rmse))
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,5))
ax1.hist(errors_rmse, bins = 20, range=(0.5, 9), edgecolor = 'black', weights= weights)
ax1.set_xlabel('Average RMSE for Each Selected Day')
ax1.set_ylabel('Percentage')
ax1.set_ylim(0, 0.27)
ax2.hist(errors_mae, bins = 20, range=(0.5, 9), edgecolor = 'black', weights= weights)
ax2.set_xlabel('Average MAE for Each Selected Day')
ax2.set_ylabel('Percentage')
ax1.set_ylim(0, 0.27)
plt.savefig('HistRMSEDays.jpeg', dpi=300)
plt.show()

In [ ]:
testScores_stations = dict()
idx = [error < 1.5*mean(errors_rmse) for error in errors_rmse]
testScores_stations, test_min_, test_mean_, test_max_  = stat_scores(test_pred_orig_dict, idx)
print("Test Stations with min, mean and max RMSE are %s, %s, %s" % (test_min_, test_mean_, test_max_))

# save predicted result to nparray

In [ ]:
# test_stations are not shuffled
outputStations = stations[stations['geohash'].isin(list(test_pred_orig_dict.keys()))]
list(outputStations['geohash'])== list(test_pred_orig_dict.keys())

In [ ]:
outputStations.to_csv('/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/deliverables/stations.csv', index=False)

np.save('/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/deliverables/selectedDates.npy', test_times)
np.save('/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/deliverables/predictions.npy', 
        np.array([x[0].numpy() for x in list(test_pred_orig_dict.values())]))
np.save('/media/fbx5002/CodeMonkey/IEE/IoT_Weather_Data/deliverables/observations.npy', 
        np.array([x[1].numpy() for x in list(test_pred_orig_dict.values())]))